# Small price bias in the cryptocurrency market. 
## A cognitive bias revealed by emotions on social networks.


## (0) Prepocessing 

In [112]:
#Importar las librerias requeridas
import numpy as np
import pandas as pd

#Handle dates
import datetime as dt
from datetime import datetime, timedelta


import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import math

#Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

from linearmodels import PanelOLS, FamaMacBeth
import statsmodels.api as sm

import numpy as np
from statsmodels.datasets import grunfeld
data = grunfeld.load_pandas().data
data.year = data.year.astype(np.int64)
from linearmodels import PanelOLS, PooledOLS
from linearmodels import FamaMacBeth
import statsmodels.api as sm
import pandas as pd

## (1) Data Preparation

In [113]:
pd.options.mode.use_inf_as_na = True

dfPANEL=pd.read_csv('data_enr_filter.csv')

dfPANEL["fecha"] = pd.to_datetime(dfPANEL["fecha"])

In [114]:
features=[#Variables endogenas
          'fecha','Fecha','Ticker',
          'TotalTwits','pPositive','pNegative','pNeutral',
          'TotalTwits_1','Positive_1', 'Neutral_1', 'Negative_1',
          #Variables exogenas
          'Price','price_1','log_price','log_price_1','Estable',
          'Volume','ActiveAddress','MarketCap','volume_units',
          'returns','return_1'
         ]


#Eliminar las monedas con pocas observaciones
remove_coins = ['BTM','SNT',  "POWR"]
             #   "WTC"
                
dfPANEL_1=dfPANEL[~dfPANEL['Ticker'].isin(remove_coins) ][features]
  
#Escalar la variable de respuesta a 100%
dfPANEL_1['fecha']=pd.to_datetime(dfPANEL_1['fecha'])
dfPANEL_1['pPOSITIVE']=dfPANEL['pPositive']*100
dfPANEL_1['pNEUTRAL']=dfPANEL['pNeutral']*100
dfPANEL_1['pNEGATIVE']=dfPANEL['pNegative']*100

In [115]:
data_model=dfPANEL_1[dfPANEL_1['fecha']>='2021-07-01'].set_index(['Ticker','fecha']).dropna().copy()
data_model['constant']=1

## (2) EDA - Descriptives

In [137]:
####NO NEEDED

## MODELLING - REGRESSION ANALYSIS

For testing the impact of cryptocurrency market and media attention and how this may affect the presence of small price bias, we conducted additional cross sectional analysis by picking two events that occured and had an impact on crytocurrency investors. Fist is the collapse of crypto exchange FTX on November 2022. The other selected event was the stable coin failure of Terra-Luna.

## Small Price Bias . Modelos tipo Roger (2018) 

$$Y_{i,t} = \alpha + \lambda PriceCat_{i,t-1}+\epsilon_{i,t} $$


### FTX Collapse
FTX (Futures Exchange) was the third-largest cryptocurrency exchange by volume as of July 2021 with over a million members (Reuters, 2021). After the collapse cryptocurrency marketcap decreased almost USD 183 billion, and total market capitalization of digital assets fall below USD 800 billion during the month of the collapse (November 2022), this according to data from TradingView, and reported by Coindesk and the Economic Times.

To conduct the cross sectional analysis we take month previous collapse and month after collapse, and perform the discrete variable regression for small price bias as defined previously.

In [116]:
pre_fx_fecha_inicio='2022-10-09'

event_fx_fecha='2022-11-08'

pos_fx_fecha_fin='2022-12-07'

#1-month before event 
data_model_pre=dfPANEL_1[(dfPANEL_1['fecha']>=pre_fx_fecha_inicio) & 
                         (dfPANEL_1['fecha']<event_fx_fecha)].set_index(
    ['Ticker','fecha']).dropna().copy()
data_model_pre['constant']=1

#1-month after event
data_model_pos=dfPANEL_1[(dfPANEL_1['fecha']>=event_fx_fecha) & 
                         (dfPANEL_1['fecha']<pos_fx_fecha_fin)].set_index(
    ['Ticker','fecha']).dropna().copy()

data_model_pre['constant']=1
data_model_pos['constant']=1

In [117]:
#Prepare data for previous
data_model_pre['cat_prev_price1']=np.where(data_model_pre['price_1']<= 1,1,0)
data_model_pre['cat_prev_price2']=np.where((data_model_pre['price_1']> 1) & (data_model_pre['price_1']<= 10),1,0)
data_model_pre['cat_prev_price3']=np.where((data_model_pre['price_1']> 10) & (data_model_pre['price_1']<= 100),1,0)
data_model_pre['cat_prev_price4']=np.where((data_model_pre['price_1']> 100) & (data_model_pre['price_1']<= 1000),1,0)
data_model_pre['cat_prev_price5']=np.where((data_model_pre['price_1']> 1000),1,0)


data_model_pre['cat_price1']=np.where(data_model_pre['Price']<= 1,1,0)
data_model_pre['cat_price2']=np.where((data_model_pre['Price']> 1) & (data_model_pre['Price']<= 10),1,0)
data_model_pre['cat_price3']=np.where((data_model_pre['Price']> 10) & (data_model_pre['Price']<= 100),1,0)
data_model_pre['cat_price4']=np.where((data_model_pre['Price']> 100) & (data_model_pre['Price']<= 1000),1,0)
data_model_pre['cat_price5']=np.where((data_model_pre['Price']> 1000),1,0)



In [118]:
#Prepare data for pos event
data_model_pos['cat_prev_price1']=np.where(data_model_pos['price_1']<= 1,1,0)
data_model_pos['cat_prev_price2']=np.where((data_model_pos['price_1']> 1) & (data_model_pos['price_1']<= 10),1,0)
data_model_pos['cat_prev_price3']=np.where((data_model_pos['price_1']> 10) & (data_model_pos['price_1']<= 100),1,0)
data_model_pos['cat_prev_price4']=np.where((data_model_pos['price_1']> 100) & (data_model_pos['price_1']<= 1000),1,0)
data_model_pos['cat_prev_price5']=np.where((data_model_pos['price_1']> 1000),1,0)


data_model_pos['cat_price1']=np.where(data_model_pos['Price']<= 1,1,0)
data_model_pos['cat_price2']=np.where((data_model_pos['Price']> 1) & (data_model_pos['Price']<= 10),1,0)
data_model_pos['cat_price3']=np.where((data_model_pos['Price']> 10) & (data_model_pos['Price']<= 100),1,0)
data_model_pos['cat_price4']=np.where((data_model_pos['Price']> 100) & (data_model_pos['Price']<= 1000),1,0)
data_model_pos['cat_price5']=np.where((data_model_pos['Price']> 1000),1,0)

#### *Previous  month to event*

In [119]:
data_model=pd.DataFrame()
data_model=data_model_pre.copy()

In [120]:
'''Positividad
'''
regfmRoger_prev_pPositive=FamaMacBeth(data_model.pPOSITIVE,data_model[['constant','cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")

regfmRoger_prev_pPositive

Dep. Variable:,pPOSITIVE,R-squared:,0.1741
Estimator:,FamaMacBeth,R-squared (Between):,0.3534
No. Observations:,1148,R-squared (Within):,-0.0098
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.1741
Time:,19:33:44,Log-likelihood,-4381.8
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,60.241
Entities:,39,P-value,0.0000
Avg Obs:,29.436,Distribution:,"F(4,1143)"
Min Obs:,8.0000,,
Max Obs:,30.000,F-statistic (robust):,129.35


In [121]:
'''Negatividad
'''
regfmRoger_prev_pNegative=FamaMacBeth(data_model.pNEGATIVE,data_model[['constant',
                                                                       'cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")
regfmRoger_prev_pNegative


Dep. Variable:,pNEGATIVE,R-squared:,0.0640
Estimator:,FamaMacBeth,R-squared (Between):,0.1385
No. Observations:,1148,R-squared (Within):,-0.0157
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.0640
Time:,19:33:44,Log-likelihood,-3710.2
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,19.550
Entities:,39,P-value,0.0000
Avg Obs:,29.436,Distribution:,"F(4,1143)"
Min Obs:,8.0000,,
Max Obs:,30.000,F-statistic (robust):,59.298


In [122]:
'''Neutralidad
'''
regfmRoger_pNeutral_prev=FamaMacBeth(data_model.pNEUTRAL,data_model[['constant','cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")

regfmRoger_pNeutral_prev

Dep. Variable:,pNEUTRAL,R-squared:,0.2142
Estimator:,FamaMacBeth,R-squared (Between):,0.3980
No. Observations:,1148,R-squared (Within):,-0.0079
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.2142
Time:,19:33:44,Log-likelihood,-4450.8
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,77.893
Entities:,39,P-value,0.0000
Avg Obs:,29.436,Distribution:,"F(4,1143)"
Min Obs:,8.0000,,
Max Obs:,30.000,F-statistic (robust):,222.98


#### *After month to event*

In [123]:
data_model=pd.DataFrame()
data_model=data_model_pos.copy()

In [124]:
'''
Positividad
'''
regfmRoger_pos_pPositive=FamaMacBeth(data_model.pPOSITIVE,data_model[['constant','cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")

regfmRoger_pos_pPositive

Dep. Variable:,pPOSITIVE,R-squared:,0.0861
Estimator:,FamaMacBeth,R-squared (Between):,0.1696
No. Observations:,1114,R-squared (Within):,-0.0054
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.0861
Time:,19:33:44,Log-likelihood,-4399.3
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,26.107
Entities:,39,P-value,0.0000
Avg Obs:,28.564,Distribution:,"F(4,1109)"
Min Obs:,12.000,,
Max Obs:,29.000,F-statistic (robust):,68.482


In [125]:
'''Neutralidad
'''
regfmRoger_pNeutral_pos=FamaMacBeth(data_model.pNEUTRAL,data_model[['constant','cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")

regfmRoger_pNeutral_pos

Dep. Variable:,pNEUTRAL,R-squared:,0.1029
Estimator:,FamaMacBeth,R-squared (Between):,0.1879
No. Observations:,1114,R-squared (Within):,-0.0109
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.1029
Time:,19:33:44,Log-likelihood,-4461.5
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,31.796
Entities:,39,P-value,0.0000
Avg Obs:,28.564,Distribution:,"F(4,1109)"
Min Obs:,12.000,,
Max Obs:,29.000,F-statistic (robust):,76.463


In [126]:
'''Negatividad
'''
regfmRoger_pos_pNegative=FamaMacBeth(data_model.pNEGATIVE,data_model[['constant',
                                                                       'cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")
regfmRoger_pos_pNegative



Dep. Variable:,pNEGATIVE,R-squared:,0.0452
Estimator:,FamaMacBeth,R-squared (Between):,0.1059
No. Observations:,1114,R-squared (Within):,-0.0043
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.0452
Time:,19:33:44,Log-likelihood,-3624.7
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,13.133
Entities:,39,P-value,0.0000
Avg Obs:,28.564,Distribution:,"F(4,1109)"
Min Obs:,12.000,,
Max Obs:,29.000,F-statistic (robust):,20.538


### Stable coin drop afer LunaTERRA (May 2022)

In [127]:
pre_luna_fecha_inicio='2022-05-01'

event_luna_fecha='2022-05-07'

pos_luna_fecha_fin='2022-05-15'

#1-month before event 
data_model_pre=dfPANEL_1[(dfPANEL_1['fecha']>=pre_luna_fecha_inicio) & 
                         (dfPANEL_1['fecha']<event_luna_fecha)].set_index(
    ['Ticker','fecha']).dropna().copy()
data_model_pre['constant']=1

#1-month after event
data_model_pos=dfPANEL_1[(dfPANEL_1['fecha']>=event_luna_fecha) & 
                         (dfPANEL_1['fecha']<pos_luna_fecha_fin)].set_index(
    ['Ticker','fecha']).dropna().copy()

data_model_pre['constant']=1
data_model_pos['constant']=1

In [128]:
#Prepare data for previous
data_model_pre['cat_prev_price1']=np.where(data_model_pre['price_1']<= 1,1,0)
data_model_pre['cat_prev_price2']=np.where((data_model_pre['price_1']> 1) & (data_model_pre['price_1']<= 10),1,0)
data_model_pre['cat_prev_price3']=np.where((data_model_pre['price_1']> 10) & (data_model_pre['price_1']<= 100),1,0)
data_model_pre['cat_prev_price4']=np.where((data_model_pre['price_1']> 100) & (data_model_pre['price_1']<= 1000),1,0)
data_model_pre['cat_prev_price5']=np.where((data_model_pre['price_1']> 1000),1,0)


data_model_pre['cat_price1']=np.where(data_model_pre['Price']<= 1,1,0)
data_model_pre['cat_price2']=np.where((data_model_pre['Price']> 1) & (data_model_pre['Price']<= 10),1,0)
data_model_pre['cat_price3']=np.where((data_model_pre['Price']> 10) & (data_model_pre['Price']<= 100),1,0)
data_model_pre['cat_price4']=np.where((data_model_pre['Price']> 100) & (data_model_pre['Price']<= 1000),1,0)
data_model_pre['cat_price5']=np.where((data_model_pre['Price']> 1000),1,0)



#Prepare data for pos event
data_model_pos['cat_prev_price1']=np.where(data_model_pos['price_1']<= 1,1,0)
data_model_pos['cat_prev_price2']=np.where((data_model_pos['price_1']> 1) & (data_model_pos['price_1']<= 10),1,0)
data_model_pos['cat_prev_price3']=np.where((data_model_pos['price_1']> 10) & (data_model_pos['price_1']<= 100),1,0)
data_model_pos['cat_prev_price4']=np.where((data_model_pos['price_1']> 100) & (data_model_pos['price_1']<= 1000),1,0)
data_model_pos['cat_prev_price5']=np.where((data_model_pos['price_1']> 1000),1,0)


data_model_pos['cat_price1']=np.where(data_model_pos['Price']<= 1,1,0)
data_model_pos['cat_price2']=np.where((data_model_pos['Price']> 1) & (data_model_pos['Price']<= 10),1,0)
data_model_pos['cat_price3']=np.where((data_model_pos['Price']> 10) & (data_model_pos['Price']<= 100),1,0)
data_model_pos['cat_price4']=np.where((data_model_pos['Price']> 100) & (data_model_pos['Price']<= 1000),1,0)
data_model_pos['cat_price5']=np.where((data_model_pos['Price']> 1000),1,0)


#### *Previous  month to event*

In [129]:
data_model=pd.DataFrame()
data_model=data_model_pre.copy()

In [130]:

'''Positividad
'''
regfmRoger_prev_pPositive=FamaMacBeth(data_model.pPOSITIVE,data_model[['constant','cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")

regfmRoger_prev_pPositive




Dep. Variable:,pPOSITIVE,R-squared:,0.0396
Estimator:,FamaMacBeth,R-squared (Between):,0.0781
No. Observations:,234,R-squared (Within):,-0.0019
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.0396
Time:,19:33:45,Log-likelihood,-908.72
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,2.3581
Entities:,39,P-value,0.0544
Avg Obs:,6.0000,Distribution:,"F(4,229)"
Min Obs:,6.0000,,
Max Obs:,6.0000,F-statistic (robust):,8.0831


In [131]:

'''Neutralidad
'''
regfmRoger_pNeutral_prev=FamaMacBeth(data_model.pNEUTRAL,data_model[['constant','cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")

regfmRoger_pNeutral_prev

Dep. Variable:,pNEUTRAL,R-squared:,0.1337
Estimator:,FamaMacBeth,R-squared (Between):,0.2586
No. Observations:,234,R-squared (Within):,0.0003
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.1337
Time:,19:33:45,Log-likelihood,-913.00
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,8.8382
Entities:,39,P-value,0.0000
Avg Obs:,6.0000,Distribution:,"F(4,229)"
Min Obs:,6.0000,,
Max Obs:,6.0000,F-statistic (robust):,294.28


In [132]:

'''Negatividad
'''
regfmRoger_prev_pNegative=FamaMacBeth(data_model.pNEGATIVE,data_model[['constant',
                                                                       'cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")
regfmRoger_prev_pNegative




Dep. Variable:,pNEGATIVE,R-squared:,0.0860
Estimator:,FamaMacBeth,R-squared (Between):,0.1415
No. Observations:,234,R-squared (Within):,-0.0036
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.0860
Time:,19:33:45,Log-likelihood,-812.08
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,5.3872
Entities:,39,P-value,0.0004
Avg Obs:,6.0000,Distribution:,"F(4,229)"
Min Obs:,6.0000,,
Max Obs:,6.0000,F-statistic (robust):,36.993


#### *After  month to event*

In [133]:
data_model=pd.DataFrame()
data_model=data_model_pos.copy()

In [134]:
'''
Positividad
'''
regfmRoger_pos_pPositive=FamaMacBeth(data_model.pPOSITIVE,data_model[['constant','cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")

regfmRoger_pos_pPositive

Dep. Variable:,pPOSITIVE,R-squared:,0.0917
Estimator:,FamaMacBeth,R-squared (Between):,0.1600
No. Observations:,312,R-squared (Within):,-0.0358
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.0917
Time:,19:33:45,Log-likelihood,-1124.5
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,7.7485
Entities:,39,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(4,307)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,33.157


In [135]:
'''Neutralidad
'''
regfmRoger_pNeutral_pos=FamaMacBeth(data_model.pNEUTRAL,data_model[['constant','cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")

regfmRoger_pNeutral_pos

Dep. Variable:,pNEUTRAL,R-squared:,0.0952
Estimator:,FamaMacBeth,R-squared (Between):,0.1637
No. Observations:,312,R-squared (Within):,-0.0612
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.0952
Time:,19:33:45,Log-likelihood,-1176.8
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,8.0764
Entities:,39,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(4,307)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,22.473


In [136]:
'''Negatividad
'''
regfmRoger_pos_pNegative=FamaMacBeth(data_model.pNEGATIVE,data_model[['constant',
                                                                       'cat_prev_price2',
                                                             'cat_prev_price3','cat_prev_price4',
                                                             'cat_prev_price5']],weights=None).fit(
                                                              debiased=True,cov_type="robust", kernel="bartlett")
regfmRoger_pos_pNegative

Dep. Variable:,pNEGATIVE,R-squared:,0.0291
Estimator:,FamaMacBeth,R-squared (Between):,0.0614
No. Observations:,312,R-squared (Within):,-0.0055
Date:,"Fri, Apr 05 2024",R-squared (Overall):,0.0291
Time:,19:33:45,Log-likelihood,-1036.9
Cov. Estimator:,Fama-MacBeth Standard Cov,,
,,F-statistic:,2.2993
Entities:,39,P-value,0.0589
Avg Obs:,8.0000,Distribution:,"F(4,307)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,10.333
